### Association Rules

In [17]:
import pandas as pd
import numpy as np
import time
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
# Load dataset
df = pd.read_csv('Groceries_dataset.csv', header=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [5]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [6]:
#checking for missing values
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [7]:
#clearing column names
df.columns = df.columns.str.strip()
#altering date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

C:\Users\lilai\AppData\Local\Temp\ipykernel_4428\488416755.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


In [10]:
#Group Items by Transaction (Member + Date)
df_grouped = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index()
df_grouped.head()

,Member_number,Date,itemDescription
0,1000,2014-06-24,"[whole milk, pastry, salty snack]"
1,1000,2015-03-15,"[sausage, whole milk, semi-finished bread, yog..."
2,1000,2015-05-27,"[soda, pickled vegetables]"
3,1000,2015-07-24,"[canned beer, misc. beverages]"
4,1000,2015-11-25,"[sausage, hygiene articles]"


In [18]:
MyEncoder = TransactionEncoder()
x = MyEncoder.fit(df_grouped).transform(df_grouped)
df = pd.DataFrame(x, columns=MyEncoder.columns_)
df.head()

,D,M,_,a,b,c,e,i,m,n,o,p,r,s,t,u
0,False,True,True,False,True,False,True,False,True,True,False,False,True,False,False,True
1,True,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False
2,True,False,False,False,False,True,True,True,True,True,True,True,True,True,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [20]:
#chop the datasets into 6 parts with different number of rows
datasets = []
row_counts = [500, 1000, 5000, 7000,df.shape[0]]
start = 0
for count in row_counts:
    end = start + count
    datasets.append(df.iloc[start:end])
    start = end
datasets.append(df)  # full dataset
dataset_labels = ['500', '1000', '5000', '7000', 'full']
